In [ ]:
import json
import pandas as pd

In [ ]:
path = "../prep_data/jsonl_v1/info_question_type-9_01082022_18092023.jsonl"
with open(path, "r", encoding="utf-8") as f:
    lines = [json.loads(line.strip()) for line in f.readlines()]

df = pd.DataFrame(lines)
df.head()

In [ ]:
df.utterance_scores.hist(bins=100)

In [ ]:
df[["id", "text"]].to_csv("/data/codes/prep_ps_pykaldi/prep_data/info_question_type-9_01082022_18092023.csv", sep="|", index=None, header=None)

#### audio processing

In [ ]:
import pandas as pd
import torchaudio
import librosa
import shutil
import json
import os

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=True)

In [ ]:
audio_dir = "/data/codes/prep_ps_pykaldi/prep_data/wav"
audio_files = os.listdir(audio_dir)
audio_files = [audio_file.split(".")[0] for audio_file in audio_files]
audio_files = set(audio_files)

In [ ]:
path = "/data/codes/prep_ps_pykaldi/prep_data/info_question_type-9_01082022_18092023.csv"
df = pd.read_csv(path, names=["id", "text"], sep="|")
df.head()

In [ ]:
df.id.parallel_apply(lambda x: str(x) in audio_files).sum()

In [ ]:
import soundfile as sf

in_dir = "/data/audio_data/prep_submission_audio/9"
out_dir = audio_dir

def copy_audio(data):
    id = data["id"]
    utt_id = data["utt_id"]
    start_time = data["start_time"]
    end_time = data["end_time"]

    in_path = f'{in_dir}/{id}.wav'
    wav, sr = torchaudio.load(in_path)

    start_frame = int(start_time * sr)
    end_frame = int(end_time * sr)

    wav = wav[0][start_frame: end_frame].numpy()

    if sr == 8000:
        return False
    
    out_path = f'{out_dir}/{id}.{utt_id}.wav'
    
    sf.write(out_path, wav, samplerate=sr)
    return True
    
extracted_data.data.parallel_apply(copy_audio)